In [33]:
##################################################################
#This is a python notebook that builds a neuralnet to predict facial emotions.
#To run it requires python, Jupyter, and the packages listed below.
#We decided to separate this from main.Rmd given the dependence on Python.
#The results of training and testing are included in main.Rmd.
#This file relies on the csv files created by lib/oversampling.py
##################################################################

import tensorflow as tf
import keras
import numpy as np
from keras.models import Model, Sequential
import keras.layers as layers
import pandas as pd
import scipy.io

# Process original fiduciary points
data_root='/../data/train_set'
points = np.ndarray(shape=(3000,156),dtype=float)
for i in range(3000):
  if i == 0:
    continue
  if i < 10:
    name = '000' + str(i)
  elif i <100:
    name = '00' + str(i)
  elif i <1000:
    name = '0' + str(i)

  point = scipy.io.loadmat(str(data_root)+'/points/'+name+'.mat')

  if 'faceCoordinatesUnwarped' in point.keys():
    point = point['faceCoordinatesUnwarped']
  elif 'faceCoordinates2' in point.keys():
    point = point['faceCoordinates2']
  points[i] = point.flatten()

matrix_df = pd.read_csv(str(data_root)+'/label.csv',dtype = str)

matrix_df["label"]= matrix_df["label"].astype(int) 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:

#Create model and train on original fiduciary points
model_a = Sequential()
model_a.add(layers.Dense(64, activation='relu', input_dim=156))
model_a.add(layers.Dense(32, activation='relu'))
model_a.add(layers.Dense(16, activation='relu'))
model_a.add(layers.Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam()
model_a.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy','AUC'])

import time
start = time.time()
history = model_a.fit(x=points,
                          y=matrix_df['label'],
                          epochs=25,
                          verbose=2,
                          validation_split=.2)
end = time.time()
print(end - start)


Epoch 1/25
75/75 - 1s - loss: 5.5659 - accuracy: 0.6862 - auc: 0.5081 - val_loss: 2.2873 - val_accuracy: 0.8017 - val_auc: 0.5000
Epoch 2/25
75/75 - 0s - loss: 1.6692 - accuracy: 0.6783 - auc: 0.4737 - val_loss: 1.1808 - val_accuracy: 0.8017 - val_auc: 0.5000
Epoch 3/25
75/75 - 0s - loss: 1.2976 - accuracy: 0.6717 - auc: 0.5134 - val_loss: 1.2842 - val_accuracy: 0.8017 - val_auc: 0.5000
Epoch 4/25
75/75 - 0s - loss: 2.8335 - accuracy: 0.6896 - auc: 0.5340 - val_loss: 1.4978 - val_accuracy: 0.8017 - val_auc: 0.5000
Epoch 5/25
75/75 - 0s - loss: 1.2086 - accuracy: 0.6908 - auc: 0.4926 - val_loss: 1.1947 - val_accuracy: 0.1983 - val_auc: 0.5000
Epoch 6/25
75/75 - 0s - loss: 0.7884 - accuracy: 0.7325 - auc: 0.4998 - val_loss: 1.2001 - val_accuracy: 0.8017 - val_auc: 0.5000
Epoch 7/25
75/75 - 0s - loss: 1.5971 - accuracy: 0.6796 - auc: 0.4923 - val_loss: 2.0989 - val_accuracy: 0.8017 - val_auc: 0.5000
Epoch 8/25
75/75 - 0s - loss: 0.8311 - accuracy: 0.7175 - auc: 0.4921 - val_loss: 0.8432 -

In [36]:
#Time model testing speed
#Note: The accuracy is not reliable here, as we are timing partially on training data
start = time.time()
model_a.evaluate(x=points[1:600],
                 y=matrix_df['label'][1:600])
end = time.time()
print(end-start)

19/19 [==============================] - 0s 1ms/step - loss: 0.4934 - accuracy: 0.8097 - auc: 0.4830
0.0839381217956543


In [37]:

#Train on SMOTE Resampled Data
data_root='/../output'
df = pd.read_csv(str(data_root)+'/feature_train_oversam.csv',dtype = float).drop(columns="Unnamed: 0")
data=(df-df.min())/(df.max()-df.min())


labs = pd.read_csv(str(data_root)+'/label_train_oversam.csv',dtype = int).drop(columns="Unnamed: 0")
labs['x'] -= 1

np.random.seed(5)
idx = np.random.permutation(data.index)
data = data.reindex(idx)
labs = labs.reindex(idx)


model_a = Sequential()
model_a.add(layers.Dense(64, activation='relu', input_dim=6006))
model_a.add(layers.Dense(32, activation='relu'))
model_a.add(layers.Dense(16, activation='relu'))
model_a.add(layers.Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam()
model_a.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy','AUC'])

import time
start = time.time()
history = model_a.fit(x=data,
                          y=labs,
                          epochs=25,
                          verbose=2,
                          validation_split=.2)
end = time.time()
print(end - start)

Epoch 1/25
97/97 - 2s - loss: 0.6884 - accuracy: 0.5487 - auc: 0.5752 - val_loss: 0.6446 - val_accuracy: 0.5736 - val_auc: 0.7621
Epoch 2/25
97/97 - 1s - loss: 0.6434 - accuracy: 0.6182 - auc: 0.6712 - val_loss: 0.6373 - val_accuracy: 0.6344 - val_auc: 0.7702
Epoch 3/25
97/97 - 1s - loss: 0.6164 - accuracy: 0.6705 - auc: 0.7202 - val_loss: 0.5665 - val_accuracy: 0.6912 - val_auc: 0.7983
Epoch 4/25
97/97 - 1s - loss: 0.5908 - accuracy: 0.6828 - auc: 0.7467 - val_loss: 0.5321 - val_accuracy: 0.7545 - val_auc: 0.8203
Epoch 5/25
97/97 - 1s - loss: 0.5723 - accuracy: 0.7038 - auc: 0.7672 - val_loss: 0.5218 - val_accuracy: 0.7687 - val_auc: 0.8264
Epoch 6/25
97/97 - 1s - loss: 0.5376 - accuracy: 0.7459 - auc: 0.8038 - val_loss: 0.5004 - val_accuracy: 0.7661 - val_auc: 0.8412
Epoch 7/25
97/97 - 1s - loss: 0.5248 - accuracy: 0.7488 - auc: 0.8106 - val_loss: 0.5359 - val_accuracy: 0.7442 - val_auc: 0.8502
Epoch 8/25
97/97 - 1s - loss: 0.5002 - accuracy: 0.7611 - auc: 0.8329 - val_loss: 0.4687 -

In [38]:
#Time model testing speed
#Note: The accuracy is not reliable here, as we are timing partially on training data
start = time.time()
model_a.evaluate(x=data[1:600],
                 y=labs[1:600])
end = time.time()
print(end-start)

19/19 [==============================] - 0s 4ms/step - loss: 0.3838 - accuracy: 0.8297 - auc: 0.9211
0.24165010452270508
